In [101]:
# Import packages.
import gym
from gym import envs
import numpy as np


# Define environment.
env = gym.make('MountainCar-v0')

rewards = []
observation = env.reset()

num_episodes = 3000

gamma = 0.99
batch_size = 50

decision_net = QNetwork(hidden_size=8, learning_rate=0.001)
target_net = QNetwork(hidden_size=8, learning_rate=0.001)

memory = Memory(max_size=30000)
actor = Actor()

for episode in range(num_episodes):
    observation = env.reset()
    state, reward, done, _ = env.step(env.action_space.sample())
    state = state.reshape(1,2)
    episode_reward = 0
    
    target_net.model.set_weights(decision_net.model.get_weights())
        
    for t, _ in enumerate(range(200)):
        action = actor.get_action(state, episode, decision_net)
        next_state, reward, done, info = env.step(action)
        next_state = next_state.reshape(1,2)
        
        if done:
            if t+2 < 200:
                reward = 1
            else:
                reward = -1
        episode_reward += reward
        
        memory.add((state, action, reward, next_state))

        state = next_state
        if done:
            if episode%100 == 0:
                print(str(episode) + " : Episode finished after {} timesteps".format(t+2))
                print('reward', reward)
                print('action', actor.get_action(state, episode, decision_net))

            # print(episode_reward)
            break
        
        # Qネットワークの重みを学習・更新する replay
        if (memory.len() > batch_size) & (t%50 == 0):
            decision_net.replay(memory, batch_size, gamma, target_net)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
0 : Episode finished after 200 timesteps
reward -1
action 1
100 : Episode finished after 200 timesteps
reward -1
action 0
200 : Episode finished after 200 timesteps
reward -1
action 0
300 : Episode finished after 200 timesteps
reward -1
action 0
400 : Episode finished after 200 timesteps
reward -1
action 1
500 : Episode finished after 200 timesteps
reward -1
action 1
600 : Episode finished after 200 timesteps
reward -1
action 0
700 : Episode finished after 200 timesteps
reward -1
action 1
800 : Episode finished after 200 timesteps
reward -1
action 1
900 : Episode finished after 200 timesteps
reward -1
action 0
1000 : Episode finished after 200 timesteps
reward -1
action 0
1100 : Episode finished after 200 timesteps
reward -1
action 1
1200 : Episode finished after 200 timesteps
reward -1
action 1
1300 : Episode finished after 200 timesteps
reward -1
action 0
1400 : Episode finished after 2

## Memory

state, action, reward, next_state

In [6]:
from collections import deque

class Memory:
    def __init__(self, max_size=1000):
        self.buffer = deque(maxlen=max_size)
        
    def add(self, experience):
        self.buffer.append(experience)
        
    def sample(self, batch_size):
        idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
        return [self.buffer[ii] for ii in idx]
    
    def len(self):
        return len(self.buffer)

In [89]:
import tensorflow as tf
from keras import backend as K
from keras import initializers
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam

In [90]:
def huberloss(y_true, y_pred):
    err = y_true - y_pred
    cond = K.abs(err) < 1.0
    L2 = 0.5 * K.square(err)
    L1 = (K.abs(err) - 0.5)
    loss = tf.where(cond, L2, L1)  # Keras does not cover where function in tensorflow :-(
    return K.mean(loss)

In [43]:
calc_target(gamma, q_value.predict(np.array([1,2]).reshape(1,2)).max(), -1)

-0.9967901922995225

In [16]:
decision_net = network()
target_net =network()

In [85]:
class QNetwork:
    def __init__(self, learning_rate=0.01, state_size=2, action_size=3, hidden_size=10):
        self.model = Sequential()
        self.model.add(Dense(hidden_size, activation='relu', input_dim=state_size,
                             kernel_initializer=initializers.TruncatedNormal(stddev=0.01)))
        self.model.add(Dense(hidden_size, activation='relu',
                             kernel_initializer=initializers.TruncatedNormal(stddev=0.01)))
        self.model.add(Dense(action_size, activation='sigmoid',
                             kernel_initializer=initializers.TruncatedNormal(stddev=0.01)))
        self.optimizer = Adam(lr=learning_rate)  # 誤差を減らす学習方法はAdam
        self.model.compile(loss=huberloss, optimizer=self.optimizer)
        
    def replay(self, memory, batch_size, gamma, target_net):
        inputs = np.zeros((batch_size, 2))
        Targets = np.zeros((batch_size, 3))
        mini_batch = memory.sample(batch_size)
        
        for i, (state_b, action_b, reward_b, next_state_b) in enumerate(mini_batch):
            inputs[i:i+1] = state_b
            target = reward_b
            
            next_action_for_estimate = decision_net.model.predict(next_state_b).argmax()
            q_value_for_estimate = target_net.model.predict(next_state_b)[0][next_action_for_estimate]
            target = reward_b + gamma * q_value_for_estimate
            
            Targets[i][action_b] = target
            
        self.model.fit(inputs, Targets, epochs=1, verbose=0)

In [72]:
class Actor:
    def get_action(self, state, episode, decision_net):   # [C]ｔ＋１での行動を返す
        # 徐々に最適行動のみをとる、ε-greedy法
        epsilon = 0.001 + 0.9 / (1.0+episode)

        if epsilon <= np.random.uniform(0, 1):
            q_values_for_action = decision_net.model.predict(state)[0]
            action = np.argmax(q_values_for_action)  # 最大の報酬を返す行動を選択する

        else:
            action = np.random.choice([0, 1])  # ランダムに行動する

        return action

In [42]:
batch_size = 20
inputs = np.zeros((batch_size, 2))
Targets = np.zeros((batch_size, 3))
mini_batch = memory.sample(batch_size)
for i, (state_b, action_b, reward_b, next_state_b) in enumerate(mini_batch):
    inputs[i:i+1] = state_b
    target = reward_b
    next_action_for_estimate = decision_net.predict(next_state_b.reshape(1,2)).argmax()
    q_value_for_estimate = target_net.predict(next_state_b.reshape(1,2))[0][next_action_for_estimate]
    target = reward_b + gamma * q_value_for_estimate
    Targets[i][action_b] = target
    
print(inputs)
Targets

[[-3.79167402e-01 -2.23429532e-03]
 [-3.77280371e-01  1.09030105e-03]
 [-4.15241087e-01  3.05925841e-03]
 [-4.48024754e-01 -6.99231946e-04]
 [-4.09522138e-01 -2.52015430e-03]
 [-3.37958122e-01  1.65150625e-04]
 [-3.81451355e-01 -2.28395226e-03]
 [-3.70677484e-01  1.10844018e-04]
 [-3.88444139e-01 -2.33847550e-03]
 [-3.75294196e-01  9.58358498e-04]
 [-4.41297322e-01 -1.75807573e-03]
 [-4.33743462e-01  1.44951371e-04]
 [-4.15179901e-01  4.49835656e-03]
 [-3.66442667e-01 -4.49703695e-03]
 [-3.71327620e-01  4.24819190e-04]
 [-3.55116305e-01 -1.55998483e-04]
 [-4.25759342e-01 -2.36673010e-03]
 [-4.20766160e-01 -2.86738320e-03]
 [-4.44636537e-01 -1.97013075e-03]
 [-3.88091843e-01  2.57874854e-03]]


array([[ 0.,  0., -1.],
       [ 0.,  0., -1.],
       [ 0.,  0., -1.],
       [ 0.,  0., -1.],
       [ 0.,  0., -1.],
       [ 0.,  0., -1.],
       [ 0.,  0., -1.],
       [-1.,  0.,  0.],
       [ 0.,  0., -1.],
       [ 0.,  0., -1.],
       [ 0.,  0., -1.],
       [ 0.,  0., -1.],
       [ 0.,  0., -1.],
       [ 0.,  0., -1.],
       [ 0.,  0., -1.],
       [ 0.,  0., -1.],
       [ 0.,  0., -1.],
       [ 0.,  0., -1.],
       [ 0.,  0., -1.],
       [ 0.,  0., -1.]])

In [83]:
np.array([np.array([1,5]), np.array([1,2]), np.array([1,2])]).reshape(3,2)

array([[1, 5],
       [1, 2],
       [1, 2]])

In [114]:
memory = Memory(max_size=1000)


In [115]:
memory.add((observation, action, reward, next_observation))

In [15]:
memory.sample(20)[0]

(array([-0.35605739, -0.00057459]), 2, -1.0, array([-0.35683629, -0.0007789 ]))